# SERP API ile Google İşletme Yorumları

In [ ]:
# Kütüphaneler
!pip install google-search-results python-dotenv

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=6dc2e0897fb4a993c7950f376b95bd833573f62725ed2f9b2fa3be5a9f921409
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [ ]:
import os
import re
from serpapi import GoogleSearch
from google.colab import userdata

# SerpApi Anahtarı
os.environ['SERPAPI_KEY'] = userdata.get('SERPAPI_KEY')

def get_location_data_v2(query):
    # Parametreleri daha geniş bir kapsama alıyoruz
    params = {
        "engine": "google_maps",
        "q": query,
        "hl": "tr",
        "gl": "tr",
        "type": "search", # Arama tipini belirginleştirdik
        "api_key": os.environ['SERPAPI_KEY']
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # 1. Senaryo: Doğrudan yerel sonuçlarda varsa
    if 'local_results' in results and len(results['local_results']) > 0:
        local = results['local_results'][0]
        data_id = local.get('data_id')
        print(f"Başarı (Local): {local['title']} | Data ID: {data_id}")
        return data_id

    # 2. Senaryo: Google Maps bazen 'place_results' altında döndürür
    elif 'place_results' in results:
        place = results['place_results']
        data_id = place.get('data_id')
        print(f"Başarı (Place): {place.get('title')} | Data ID: {data_id}")
        return data_id

    # 3. Senaryo: Hala bulunamadıysa sorguyu sadeleştirip tekrar dene
    else:
        print("İlk denemede bulunamadı, alternatif sorgu deneniyor...")
        params["q"] = "Konya Şehir Hastanesi, Karatay" # Daha spesifik adres
        search = GoogleSearch(params)
        results = search.get_dict()

        # Tekrar kontrol
        res = results.get('local_results', [{}])[0]
        data_id = res.get('data_id')
        if data_id:
            print(f"Başarı (Alternatif): {res.get('title')} | Data ID: {data_id}")
            return data_id

    print("Hata: Yer hala bulunamadı. Lütfen SerpApi dashboard'dan 'google_maps' sorgusunu manuel test edin.")
    return None

# Kullanım
target_query = "Konya Şehir Hastanesi"
data_id = get_location_data_v2(target_query)

Başarı (Place): Konya Şehir Hastanesi | Data ID: 0x14d09ac8b9fb364d:0x1ba5a9b27477a66c


In [ ]:
import pandas as pd
import time

def get_reviews_refined(target_data_id, max_pages=20):
    if not target_data_id:
        print("Hata: data_id bulunamadı. Lütfen önce Hücre 1'i çalıştırın.")
        return None

    all_reviews = []
    next_token = None
    page_count = 0

    print(f"Veri çekme işlemi başlatıldı (Max Sayfa: {max_pages})...")

    while page_count < max_pages:
        params = {
            "engine": "google_maps_reviews",
            "data_id": target_data_id,
            "hl": "tr",
            "gl": "tr",
            "sort_by": "newestFirst",
            "api_key": os.environ['SERPAPI_KEY']
        }

        if next_token:
            params["next_page_token"] = next_token

        search = GoogleSearch(params)
        results = search.get_dict()

        if "error" in results:
            print(f"API Hatası: {results['error']}")
            break

        reviews = results.get('reviews', [])
        if not reviews:
            print("Çekilecek başka yorum kalmadı.")
            break

        all_reviews.extend(reviews)

        # Bir sonraki sayfa için token kontrolü
        pagination = results.get('serpapi_pagination', {})
        next_token = pagination.get('next_page_token')

        page_count += 1
        print(f"Sayfa {page_count} çekildi. Toplam yorum sayısı: {len(all_reviews)}")

        if not next_token:
            break

        time.sleep(1) # Rate limit koruması

    # Veriyi düzenleme (Nested JSON'ları sütunlara ayırma)
    if all_reviews:
        df = pd.json_normalize(all_reviews)
        # Sütun isimlerini daha okunabilir yapalım (varsa)
        rename_map = {
            'user.name': 'Kullanıcı',
            'rating': 'Puan',
            'snippet': 'Yorum',
            'date': 'Tarih',
            'user.reviews': 'Kullanıcı_Top_Yorum_Sayısı'
        }
        df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
        return df
    return pd.DataFrame()

# Çalıştır
df_konya = get_reviews_refined(data_id)

if df_konya is not None and not df_konya.empty:
    # İlk 5 satırı göster
    print("\n--- Çekilen Verilerin Önizlemesi ---")
    display(df_konya[['Kullanıcı', 'Puan', 'Yorum', 'Tarih']].head())

    # CSV olarak dışa aktar
    csv_name = "konya_sehir_hastanesi_yorumlar.csv"
    df_konya.to_csv(csv_name, index=False, encoding='utf-8-sig')
    print(f"\nİşlem tamam! {len(df_konya)} yorum '{csv_name}' dosyasına kaydedildi.")
else:
    print("Veri çekilemedi.")

Veri çekme işlemi başlatıldı (Max Sayfa: 20)...
Sayfa 1 çekildi. Toplam yorum sayısı: 8
Sayfa 2 çekildi. Toplam yorum sayısı: 18
Sayfa 3 çekildi. Toplam yorum sayısı: 28
Sayfa 4 çekildi. Toplam yorum sayısı: 38
Sayfa 5 çekildi. Toplam yorum sayısı: 48
Sayfa 6 çekildi. Toplam yorum sayısı: 58
Sayfa 7 çekildi. Toplam yorum sayısı: 68
Sayfa 8 çekildi. Toplam yorum sayısı: 78
Sayfa 9 çekildi. Toplam yorum sayısı: 88
Sayfa 10 çekildi. Toplam yorum sayısı: 98
Sayfa 11 çekildi. Toplam yorum sayısı: 108
Sayfa 12 çekildi. Toplam yorum sayısı: 118
Sayfa 13 çekildi. Toplam yorum sayısı: 128
Sayfa 14 çekildi. Toplam yorum sayısı: 138
Sayfa 15 çekildi. Toplam yorum sayısı: 148
Sayfa 16 çekildi. Toplam yorum sayısı: 158
Sayfa 17 çekildi. Toplam yorum sayısı: 168
Sayfa 18 çekildi. Toplam yorum sayısı: 178
Sayfa 19 çekildi. Toplam yorum sayısı: 188
Sayfa 20 çekildi. Toplam yorum sayısı: 198

--- Çekilen Verilerin Önizlemesi ---


,Kullanıcı,Puan,Yorum,Tarih
0,Sedat Çeşen,5.0,Kadın Doğum poliklinikliğinde hasta mahremiyet...,bir saat önce
1,Ismet Tekin,1.0,"Mekan çok güzel! İşleyiş berbat! Bukadar kötü,...",6 saat önce
2,Emre,1.0,Gerçekten hizmet konusunda rezil bir hastane a...,2 gün önce
3,Refik Can Sunal,5.0,Gecırdıgım trafık kazasında ayagımda parcalı k...,2 gün önce
4,Hamza Alrajeh,5.0,NaN,bir hafta önce



İşlem tamam! 198 yorum 'konya_sehir_hastanesi_yorumlar.csv' dosyasına kaydedildi.


In [ ]:
df_konya.head(20)

,link,Puan,Tarih,iso_date,iso_date_of_last_edit,source,review_id,Kullanıcı,user.link,user.contributor_id,user.thumbnail,user.local_guide,Kullanıcı_Top_Yorum_Sayısı,user.photos,Yorum,extracted_snippet.original,images,extracted_snippet.translated
0,https://www.google.com/maps/reviews/data=!4m8!...,5.0,2 gün önce,2026-01-26T13:22:54Z,2026-01-26T13:22:54Z,Google,Ci9DQUlRQUNvZENodHljRjlvT2s1RlFWUjNha0ZsTmw5RV...,Hamza Alrajeh,https://www.google.com/maps/contrib/1135319592...,113531959273176418301,https://lh3.googleusercontent.com/a-/ALV-UjUh4...,True,0,14,NaN,NaN,NaN,NaN
1,https://www.google.com/maps/reviews/data=!4m8!...,5.0,3 gün önce,2026-01-25T08:33:32Z,2026-01-25T08:34:13Z,Google,Ci9DQUlRQUNvZENodHljRjlvT2xwa2FrVXpRblpDY0hWcW...,Adem ILDIRAR,https://www.google.com/maps/contrib/1054805835...,105480583558887123827,https://lh3.googleusercontent.com/a/ACg8ocL2A6...,True,7,3,"Biraz karışık ancak temiz, doktorları ilgili b...","Biraz karışık ancak temiz, doktorları ilgili b...",NaN,NaN
2,https://www.google.com/maps/reviews/data=!4m8!...,5.0,6 gün önce,2026-01-22T14:41:04Z,2026-01-22T14:41:04Z,Google,Ci9DQUlRQUNvZENodHljRjlvT25SV2IxRkxVMmRZY0Voel...,ertan taskin,https://www.google.com/maps/contrib/1026347405...,102634740576647518211,https://lh3.googleusercontent.com/a-/ALV-UjVSQ...,True,13,4,NaN,NaN,NaN,NaN
3,https://www.google.com/maps/reviews/data=!4m8!...,1.0,6 gün önce,2026-01-22T13:43:53Z,2026-01-22T13:43:53Z,Google,Ci9DQUlRQUNvZENodHljRjlvT2xsRVdUQkVVR1JTWWxCbW...,Hilal Güreş,https://www.google.com/maps/contrib/1134828251...,113482825108369792909,https://lh3.googleusercontent.com/a-/ALV-UjVR5...,True,1,0,Hiç beğenmedim. İlk kez gittim Konya da yeniyi...,Hiç beğenmedim. İlk kez gittim Konya da yeniyi...,NaN,NaN
4,https://www.google.com/maps/reviews/data=!4m8!...,5.0,bir hafta önce,2026-01-21T15:07:13Z,2026-01-21T15:07:13Z,Google,Ci9DQUlRQUNvZENodHljRjlvT21KMUxYWTNVbFpMVDE5Rm...,Hayrullah Kurt,https://www.google.com/maps/contrib/1052849785...,105284978536815446134,https://lh3.googleusercontent.com/a/ACg8ocLtVd...,True,0,0,NaN,NaN,NaN,NaN
5,https://www.google.com/maps/reviews/data=!4m8!...,5.0,bir hafta önce,2026-01-20T12:41:42Z,2026-01-20T12:41:42Z,Google,Ci9DQUlRQUNvZENodHljRjlvT2pac1NUSjZTV2N5UlZJM1...,Ozgur Dogan,https://www.google.com/maps/contrib/1109598637...,110959863791153075180,https://lh3.googleusercontent.com/a/ACg8ocIINZ...,True,1,0,İsmim Özgür Doğan bundan yaklasık 5 ay önce iş...,İsmim Özgür Doğan bundan yaklasık 5 ay önce iş...,NaN,NaN
6,https://www.google.com/maps/reviews/data=!4m8!...,1.0,bir hafta önce,2026-01-16T14:04:41Z,2026-01-16T14:06:30Z,Google,Ci9DQUlRQUNvZENodHljRjlvT2s4eVNuWjZPVGQyZFRGaV...,Nebahat Oluktaş,https://www.google.com/maps/contrib/1023836434...,102383643465131658898,https://lh3.googleusercontent.com/a-/ALV-UjW79...,True,3,0,Kadın hastalıkları için randevu aldım şehir ha...,Kadın hastalıkları için randevu aldım şehir ha...,NaN,NaN
7,https://www.google.com/maps/reviews/data=!4m8!...,1.0,2 hafta önce,2026-01-14T06:58:43Z,2026-01-14T06:58:43Z,Google,Ci9DQUlRQUNvZENodHljRjlvT2tGSVQyNVlibGhpUjFsUU...,Sila Işık,https://www.google.com/maps/contrib/1060736126...,106073612605423142961,https://lh3.googleusercontent.com/a/ACg8ocIXM3...,True,1,0,Hastaneye geliyorum randevu iptal edildi diyor...,Hastaneye geliyorum randevu iptal edildi diyor...,NaN,NaN
8,https://www.google.com/maps/reviews/data=!4m8!...,1.0,2 hafta önce düzenlendi,2026-01-10T03:49:00Z,2026-01-11T07:29:42Z,Google,Ci9DQUlRQUNvZENodHljRjlvT2xKT01rbFhOakpzVGswME...,Saliha Mavi,https://www.google.com/maps/contrib/1067051876...,106705187674573416932,https://lh3.googleusercontent.com/a/ACg8ocLEGP...,True,2,5,Çocuk yan dallar servisinde bebeğim yatıyor. O...,Çocuk yan dallar servisinde bebeğim yatıyor. O...,[https://lh3.googleusercontent.com/geougc-cs/A...,NaN
9,https://www.google.com/maps/reviews/data=!4m8!...,1.0,2 hafta önce,2026-01-10T02:03:39Z,2026-01-10T02:03:39Z,Google,Ci9DQUlRQUNvZENodHljRjlvT25GWlFXczJaR3R5WkhKRm...,melike melike,https://

In [ ]:
# 1 veya 2 puan verilmiş yorumları filtrele
olumsuz_yorumlar = df_konya[df_konya['Puan'] <= 2][['Kullanıcı', 'Puan', 'Yorum']]
print(f"Toplam {len(olumsuz_yorumlar)} adet olumsuz yorum tespit edildi.")
display(olumsuz_yorumlar.head())

Toplam 53 adet olumsuz yorum tespit edildi.


,Kullanıcı,Puan,Yorum
3,Hilal Güreş,1.0,Hiç beğenmedim. İlk kez gittim Konya da yeniyi...
6,Nebahat Oluktaş,1.0,Kadın hastalıkları için randevu aldım şehir ha...
7,Sila Işık,1.0,Hastaneye geliyorum randevu iptal edildi diyor...
8,Saliha Mavi,1.0,Çocuk yan dallar servisinde bebeğim yatıyor. O...
9,melike melike,1.0,Geç acilde ORTOPEDİ asistanı SEFA İBİŞ adlı ş...


In [ ]:
len(df_konya)

98

In [ ]:
olumlu_yorumlar = df_konya[df_konya['Puan'] > 3][['Kullanıcı', 'Puan', 'Yorum']]
display(olumlu_yorumlar.head())

,Kullanıcı,Puan,Yorum
0,Hamza Alrajeh,5.0,NaN
1,Adem ILDIRAR,5.0,"Biraz karışık ancak temiz, doktorları ilgili b..."
2,ertan taskin,5.0,NaN
4,Hayrullah Kurt,5.0,NaN
5,Ozgur Dogan,5.0,İsmim Özgür Doğan bundan yaklasık 5 ay önce iş...


In [ ]:
olumlu_yorumlar['Yorum'][5]

'İsmim Özgür Doğan bundan yaklasık 5 ay önce iş kazası yaşadım sağ kalca kemiğim bacağım kırıldı ayrıca sol kolum bileğimden kırıldı sağ olsun Mehmet Kılıç hocamın gayretleriyle ağır bir ameliyatı basarıyla sonuçlandırdı kendisine müteşekkirim sakat kalma ihtimalinin olduğu cok zor bi ameliyattı sehir hastanesi personeline de ayrıca teşekkürlerimi bildiririm Allah hepsinden razı olsun hocamada yine teşekkür ediyorum'

In [ ]:
len(olumlu_yorumlar)

41

In [ ]:
from google import genai
from google.genai import types

In [ ]:
from google.colab import userdata
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY_1')

In [ ]:
client=genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
query=olumlu_yorumlar['Yorum'][5]

In [ ]:
type(query)

str

In [ ]:
query=olumsuz_yorumlar['Yorum'][6]

In [ ]:
SYSTEM_PROMPTU="""
Sana verilen metnmi değerlendir sonucu -1 +1 aralığında puanla. -1 Çok olumsuz, 0:Notr, +1 Çok olumlu. Dönüğt olarak bu üç rakamdan birini ver. İncelenecek metin:
"""

In [ ]:
query="""
Randevu almak için muhteşem randevu sistemiyle uğraşmak zorunda kaldım. Gerçekten harika bir deneyimdi.
"""

In [ ]:
query="""
Hastane gerçekten beni şaşırttı. Bu kadarını beklemiyordum. Ömrüm yeterse bir hekim görebilirim sanırım.
"""

In [ ]:
query="""
30 dönüm arazi üzerine kurulmuştur.
"""

In [ ]:
response=client.models.generate_content(
    model='gemini-2.5-flash',
        config=types.GenerateContentConfig(
        system_instruction=SYSTEM_PROMPTU,
        temperature=0.1),
    contents=query,
)
print(response.text)

0
